In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

import gc

In [ ]:
%%time
train = pd.read_feather('../input/amexfeather/train_data.ftr')
train.head()

In [ ]:
train.shape

In [ ]:
train =  (train
            .groupby('customer_ID')
            .tail(1)
            .set_index('customer_ID', drop=True)
            .sort_index()
            .drop(['S_2'], axis='columns'))

In [ ]:
train.shape

In [ ]:
_ = gc.collect()

In [ ]:
cols = train.columns.to_list()
category_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
numerical_cols = [col for col in cols if col not in category_cols + ['target']]

In [ ]:
X = train[category_cols + numerical_cols]
y = train['target']

X.shape, y.shape

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
X[category_cols] = enc.fit_transform(X[category_cols])
_ = gc.collect()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()


In [ ]:
# Fit
eval_set = [(X_test, y_test)]
xgb_cl.fit(X_train, y_train,early_stopping_rounds=50, eval_metric="logloss", eval_set=eval_set, verbose=True)

In [ ]:
import pickle
filename = 'Xgboost.sav'
pickle.dump(xgb_cl, open(filename, 'wb'))

In [ ]:
y_pred = pd.DataFrame(y_test.copy(deep=True))
y_pred = y_pred.rename(columns={'target':'prediction'})
y_pred

In [ ]:
y_pred['prediction'] = xgb_cl.predict_proba(X_test)[:,1]
y_pred

In [ ]:
y_test = pd.DataFrame(y_test)

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
%%time
amex_metric(y_test, y_pred)

In [ ]:
del train, X, y, X_test, X_train, y_train, y_test, y_pred
_ = gc.collect()

In [ ]:
%%time
test = pd.read_feather('../input/amexfeather/test_data.ftr')
test.head()

In [ ]:
test =  (
    test
    .groupby('customer_ID')
    .tail(1)
    .set_index('customer_ID', drop=True)
    .sort_index()
    .drop(['S_2'], axis='columns')
)

In [ ]:

test[category_cols] = enc.transform(test[category_cols])
_ = gc.collect()

In [ ]:
import pickle
loaded_model = pickle.load(open('../input/american-express-credit-xgboost-model/Xgboost.sav', 'rb'))

In [ ]:
# Predict
test["prediction"] = loaded_model.predict_proba(test)[:,1]

In [ ]:
test["prediction"].to_csv("submission.csv", index=True)